In [0]:
!pip install tensorflow==2.0.0

import tensorflow as tf
print(tf.__version__)

# Custom Loss Function

In [0]:
def create_huber(threshold = 1.0):
  def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)
  return huber_fn

# model.compile(loss=create_huber(2.0), optimizer="nadam")

In [0]:
from tensorflow import keras

class HuberLoss(keras.losses.Loss):
  def __init__(self, threshold=1.0, **kwargs):
      self.threshold = threshold
      super().__init__(**kwargs)
  def call(self, y_true, y_pred):
      error = y_true - y_pred
      is_small_error = tf.abs(error) < self.threshold
      squared_loss = tf.square(error) / 2
      linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
      return tf.where(is_small_error, squared_loss, linear_loss)
  def get_config(self):
      base_config = super().get_config()
      return {**base_config, "threshold": self.threshold}

# Other Custom Implementations

In [0]:
def my_softplus(z): # return value is just tf.nn.softplus(z)
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [0]:
layer = keras.layers.Dense(30, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

# Custom Layers

In [0]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))